In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import calendar
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import xgboost as xgb

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train=pd.read_csv("train.csv",nrows=2000000)
print("Shape of Training Data",train.shape)
test=pd.read_csv("test.csv")
print("Shape of Testing Data", test.shape)
train.loc[train['fare_amount']<0].shape

Shape of Training Data (2000000, 8)
Shape of Testing Data (9914, 7)


(77, 8)

In [3]:
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]

In [4]:
def clean_data(data):
    boundary={'min_lng':-74.263242,
              'min_lat':40.573143,
              'max_lng':-72.986532, 
              'max_lat':41.709555}
    
    data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
    data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
    data['pickup_hour']=data['pickup_datetime'].apply(lambda x:x.hour)
    data['pickup_day_of_week']=data['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
    data['pickup_month']=data['pickup_datetime'].apply(lambda x:x.month)
    data['pickup_year']=data['pickup_datetime'].apply(lambda x:x.year)
    if 'fare_amount' in data.columns:
        data=data[data['fare_amount']>=0]
        data.loc[~((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
            (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
            (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
            (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=1
        data.loc[((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
            (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
            (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
            (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=0

    #print("Outlier vs Non Outlier Counts")
    #print(data['is_outlier_loc'].value_counts())

    # Let us drop rows, where location is outlier
        data=data.loc[data['is_outlier_loc']==0]
        data.drop(['is_outlier_loc'],axis=1,inplace=True)
    
    data=data[data['passenger_count']<=8]
    data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
    return data

In [5]:
train=clean_data(train)
test=clean_data(test)
print(train.shape)
print(test.shape)

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(1957458, 13)
(9914, 12)


In [6]:
def processDataForModelling(data,target,drop_cols,is_train=True,split=0.25):
    data_1=data.drop(drop_cols,axis=1)
    # One hot Encoding
    data_1=pd.get_dummies(data_1)
    if is_train==True:
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1

In [7]:
X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=True,split=0.1)

Shape of Training Features (1761712, 10)
Shape of Validation Features  (195746, 10)


In [8]:
test_data=processDataForModelling(test,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=False)

Shape of Test Data (9914, 10)


In [9]:
avg_fare=round(np.mean(y_train),2)
avg_fare

11.32

# Simple baseline model

In [10]:
baseline_pred=np.repeat(avg_fare,y_test.shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, y_test))
print("Basline RMSE of Validation data :",baseline_rmse)

Basline RMSE of Validation data : 9.642490185825663


# LinReg Model

In [11]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=np.round(lm.predict(X_test),2)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)

RMSE for Linear Regression is  8.191149460055973


# Random Forest Model

In [12]:
rf = RandomForestRegressor(n_estimators = 10, random_state = 883,n_jobs=-1)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=883, verbose=0, warm_start=False)

In [13]:
rf_pred= rf.predict(X_test)
rf_rmse=np.sqrt(mean_squared_error(rf_pred, y_test))
print("RMSE for Random Forest is ",rf_rmse)

RMSE for Random Forest is  3.991460608341765


In [14]:
# Using LightGBM algorithm

In [15]:
train_data=lgb.Dataset(X_train,label=y_train)

In [16]:
param = {'num_leaves':31, 'num_trees':5000, 'objective':'regression'}
param['metric'] = 'l2_root'

In [17]:
num_round=5000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=10,verbose_eval=20, early_stopping_rounds=20,stratified=False)

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	cv_agg's rmse: 5.08278 + 0.0934834
[40]	cv_agg's rmse: 4.54805 + 0.105815
[60]	cv_agg's rmse: 4.34493 + 0.114242
[80]	cv_agg's rmse: 4.25264 + 0.115786
[100]	cv_agg's rmse: 4.19623 + 0.117897
[120]	cv_agg's rmse: 4.15468 + 0.118655
[140]	cv_agg's rmse: 4.1218 + 0.120755
[160]	cv_agg's rmse: 4.09327 + 0.119202
[180]	cv_agg's rmse: 4.07281 + 0.12147
[200]	cv_agg's rmse: 4.05325 + 0.121248
[220]	cv_agg's rmse: 4.03676 + 0.122824
[240]	cv_agg's rmse: 4.02188 + 0.122723
[260]	cv_agg's rmse: 4.0102 + 0.122141
[280]	cv_agg's rmse: 4.0009 + 0.122785
[300]	cv_agg's rmse: 3.99187 + 0.122125
[320]	cv_agg's rmse: 3.98446 + 0.121258
[340]	cv_agg's rmse: 3.9759 + 0.121333
[360]	cv_agg's rmse: 3.96868 + 0.121491
[380]	cv_agg's rmse: 3.96195 + 0.12139
[400]	cv_agg's rmse: 3.9576 + 0.122478
[420]	cv_agg's rmse: 3.95238 + 0.122116
[440]	cv_agg's rmse: 3.94763 + 0.122401
[460]	cv_agg's rmse: 3.94354 + 0.122092
[480]	cv_agg's rmse: 3.93956 + 0.122357
[500]	cv_agg's rmse: 3.93707 + 0.122438
[520]	cv_a

In [18]:
lgb_bst=lgb.train(param,train_data,len(cv_results['rmse-mean']))

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [19]:
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM is ",lgb_rmse)

RMSE for Light GBM is  3.887381180779112


# Creating a xgboost model

In [20]:
dtrain = xgb.DMatrix(X_train,label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [22]:
xgb_param = {'objective':'reg:linear','eval_metric':'rmse'}
xgb_cv=xgb.cv(xgb_param, dtrain, num_boost_round=200, nfold=5,early_stopping_rounds=20)

In [23]:
nrounds=xgb_cv.shape[0]

In [24]:
xbg_model=xgb.train(params={'objective':'reg:linear','eval_metric':'rmse'}
                    ,dtrain=dtrain,num_boost_round=nrounds)

In [25]:
xgb_pred=xbg_model.predict(dtest)
xgb_rmse=np.sqrt(mean_squared_error(xgb_pred, y_test))
print("RMSE for XGBOOST is ",xgb_rmse)

RMSE for XGBOOST is  3.8479755782680103


In [26]:
model_pred=pd.DataFrame()
model_pred['model_name']=['Linear Regression','Random Forest','Light GBM','XGBOOST']
model_pred['test_rmse']=[lm_rmse,rf_rmse,lgb_rmse,xgb_rmse]

In [27]:
lm_train_rmse=np.sqrt(mean_squared_error(lm.predict(X_train), y_train))
rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(X_train),y_train))
lgb_train_rmse=np.sqrt(mean_squared_error(lgb_bst.predict(X_train),y_train))
xgb_train_rmse=np.sqrt(mean_squared_error(xbg_model.predict(dtrain),y_train))

model_pred['train_rmse']=[lm_train_rmse,rf_train_rmse,lgb_train_rmse,xgb_train_rmse]

In [28]:
model_pred['variance']=model_pred['train_rmse'] - model_pred['test_rmse']
model_pred

,model_name,test_rmse,train_rmse,variance
0,Linear Regression,8.191149,8.218900,0.027750
1,Random Forest,3.991461,1.718659,-2.272802
2,Light GBM,3.887381,3.008072,-0.879309
3,XGBOOST,3.847976,3.386853,-0.461123


## random forest has very high difference in the train and test RMSE(though train rmse is the minimum).we will thus consider LightGBM as the model and add features and tune this model.

In [30]:
nyc_airports={'JFK':{'min_lng':-73.8352,
     'min_lat':40.6195,
     'max_lng':-73.7401, 
     'max_lat':40.6659},
              
    'EWR':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

        },
    'LaGuardia':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
        
    }
    
}
def isAirport(latitude,longitude,airport_name='JFK'):
    
    if latitude>=nyc_airports[airport_name]['min_lat'] and latitude<=nyc_airports[airport_name]['max_lat'] and longitude>=nyc_airports[airport_name]['min_lng'] and longitude<=nyc_airports[airport_name]['max_lng']:
        return 1
    else:
        return 0

In [31]:
X_train['is_pickup_la_guardia']=X_train.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'LaGuardia'),axis=1)
X_train['is_dropoff_la_guardia']=X_train.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'LaGuardia'),axis=1)
X_train['is_pickup_EWR']=X_train.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'EWR'),axis=1)
X_train['is_dropoff_EWR']=X_train.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'EWR'),axis=1)
X_train['is_pickup_JFK']=X_train.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK'),axis=1)
X_train['is_dropoff_JFK']=X_train.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'JFK'),axis=1)




X_test['is_pickup_la_guardia']=X_test.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'LaGuardia'),axis=1)
X_test['is_dropoff_la_guardia']=X_test.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'LaGuardia'),axis=1)
X_test['is_pickup_EWR']=X_test.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'EWR'),axis=1)
X_test['is_dropoff_EWR']=X_test.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'EWR'),axis=1)
X_test['is_pickup_JFK']=X_test.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK'),axis=1)
X_test['is_dropoff_JFK']=X_test.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'JFK'),axis=1)


test['is_pickup_la_guardia']=test.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'LaGuardia'),axis=1)
test['is_dropoff_la_guardia']=test.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'LaGuardia'),axis=1)
test['is_pickup_EWR']=test.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'EWR'),axis=1)
test['is_dropoff_EWR']=test.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'EWR'),axis=1)
test['is_pickup_JFK']=test.apply(lambda row:isAirport(row['pickup_latitude'],row['pickup_longitude'],'JFK'),axis=1)
test['is_dropoff_JFK']=test.apply(lambda row:isAirport(row['dropoff_latitude'],row['dropoff_longitude'],'JFK'),axis=1)

In [32]:
nyc_boroughs={
    'manhattan':{
        'min_lng':-74.0479,
        'min_lat':40.6829,
        'max_lng':-73.9067,
        'max_lat':40.8820
    },
    
    'queens':{
        'min_lng':-73.9630,
        'min_lat':40.5431,
        'max_lng':-73.7004,
        'max_lat':40.8007

    },

    'brooklyn':{
        'min_lng':-74.0421,
        'min_lat':40.5707,
        'max_lng':-73.8334,
        'max_lat':40.7395

    },

    'bronx':{
        'min_lng':-73.9339,
        'min_lat':40.7855,
        'max_lng':-73.7654,
        'max_lat':40.9176

    },

    'staten_island':{
        'min_lng':-74.2558,
        'min_lat':40.4960,
        'max_lng':-74.0522,
        'max_lat':40.6490

    }
}

In [33]:
def getBorough(lat,lng):
    
    locs=nyc_boroughs.keys()
    for loc in locs:
        if lat>=nyc_boroughs[loc]['min_lat'] and lat<=nyc_boroughs[loc]['max_lat'] and lng>=nyc_boroughs[loc]['min_lng'] and lng<=nyc_boroughs[loc]['max_lng']:
            return loc
    return 'others'
X_train['pickup_borough']=X_train.apply(lambda row:getBorough(row['pickup_latitude'],row['pickup_longitude']),axis=1)
X_train['dropoff_borough']=X_train.apply(lambda row:getBorough(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)
X_test['pickup_borough']=X_test.apply(lambda row:getBorough(row['pickup_latitude'],row['pickup_longitude']),axis=1)
X_test['dropoff_borough']=X_test.apply(lambda row:getBorough(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

test['pickup_borough']=test.apply(lambda row:getBorough(row['pickup_latitude'],row['pickup_longitude']),axis=1)
test['dropoff_borough']=test.apply(lambda row:getBorough(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

X_train=pd.get_dummies(X_train)
X_test=pd.get_dummies(X_test)
test=pd.get_dummies(test)

In [34]:
lower_manhattan_boundary={'min_lng': -74.0194,
                          'min_lat':40.6997,
                          'max_lng':-73.9716,
                          'max_lat':40.7427}

def isLowerManhattan(lat,lng):
    if lat>=lower_manhattan_boundary['min_lat'] and lat<=lower_manhattan_boundary['max_lat'] and lng>=lower_manhattan_boundary['min_lng'] and lng<=lower_manhattan_boundary['max_lng']:
        return 1
    else:
        return 0

In [35]:
X_train['is_pickup_lower_manhattan']=X_train.apply(lambda row:isLowerManhattan(row['pickup_latitude'],row['pickup_longitude']),axis=1)
X_train['is_dropoff_lower_manhattan']=X_train.apply(lambda row:isLowerManhattan(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [36]:
X_test['is_pickup_lower_manhattan']=X_test.apply(lambda row:isLowerManhattan(row['pickup_latitude'],row['pickup_longitude']),axis=1)
X_test['is_dropoff_lower_manhattan']=X_test.apply(lambda row:isLowerManhattan(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

test['is_pickup_lower_manhattan']=test.apply(lambda row:isLowerManhattan(row['pickup_latitude'],row['pickup_longitude']),axis=1)
test['is_dropoff_lower_manhattan']=test.apply(lambda row:isLowerManhattan(row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [39]:
def distance(lat1,lon1,lat2,lon2):
    p = 0.017453292519943295 
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [40]:
lgr=(-73.8733, 40.7746)
jfk=(-73.7900, 40.6437)
ewr=(-74.1843, 40.6924)

test['pickup_distance_jfk']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],jfk[1],jfk[0]),axis=1)
test['dropoff_distance_jfk']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],jfk[1],jfk[0]),axis=1)
test['pickup_distance_ewr']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],ewr[1],ewr[0]),axis=1)
test['dropoff_distance_ewr']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],ewr[1],ewr[0]),axis=1)
test['pickup_distance_laguardia']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],lgr[1],lgr[0]),axis=1)
test['dropoff_distance_laguardia']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],lgr[1],lgr[0]),axis=1)

X_train['pickup_distance_jfk']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],jfk[1],jfk[0]),axis=1)
X_train['dropoff_distance_jfk']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],jfk[1],jfk[0]),axis=1)
X_train['pickup_distance_ewr']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],ewr[1],ewr[0]),axis=1)
X_train['dropoff_distance_ewr']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],ewr[1],ewr[0]),axis=1)
X_train['pickup_distance_laguardia']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],lgr[1],lgr[0]),axis=1)
X_train['dropoff_distance_laguardia']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],lgr[1],lgr[0]),axis=1)

X_test['pickup_distance_jfk']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],jfk[1],jfk[0]),axis=1)
X_test['dropoff_distance_jfk']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],jfk[1],jfk[0]),axis=1)
X_test['pickup_distance_ewr']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],ewr[1],ewr[0]),axis=1)
X_test['dropoff_distance_ewr']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],ewr[1],ewr[0]),axis=1)
X_test['pickup_distance_laguardia']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],lgr[1],lgr[0]),axis=1)
X_test['dropoff_distance_laguardia']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],lgr[1],lgr[0]),axis=1)

In [41]:
manhattan=(-73.9664, 40.7909)
queens=(-73.8317, 40.7038)
brooklyn=(-73.9489, 40.6551)
bronx=(-73.8568, 40.8572)
staten_island=(-74.1540, 40.5725)




test['pickup_distance_manhattan']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],manhattan[1],manhattan[0]),axis=1)
test['pickup_distance_queens']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],queens[1],queens[0]),axis=1)
test['pickup_distance_brooklyn']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],brooklyn[1],brooklyn[0]),axis=1)
test['pickup_distance_bronx']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],bronx[1],bronx[0]),axis=1)
test['pickup_distance_statenisland']=test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],staten_island[1],staten_island[0]),axis=1)





test['dropoff_distance_manhattan']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],manhattan[1],manhattan[0]),axis=1)
test['dropoff_distance_queens']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],queens[1],queens[0]),axis=1)
test['dropoff_distance_brooklyn']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],brooklyn[1],brooklyn[0]),axis=1)
test['dropoff_distance_bronx']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],bronx[1],bronx[0]),axis=1)
test['dropoff_distance_statenisland']=test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],staten_island[1],staten_island[0]),axis=1)


X_train['pickup_distance_manhattan']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],manhattan[1],manhattan[0]),axis=1)
X_train['pickup_distance_queens']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],queens[1],queens[0]),axis=1)
X_train['pickup_distance_brooklyn']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],brooklyn[1],brooklyn[0]),axis=1)
X_train['pickup_distance_bronx']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],bronx[1],bronx[0]),axis=1)
X_train['pickup_distance_statenisland']=X_train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],staten_island[1],staten_island[0]),axis=1)

X_train['dropoff_distance_manhattan']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],manhattan[1],manhattan[0]),axis=1)
X_train['dropoff_distance_queens']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],queens[1],queens[0]),axis=1)
X_train['dropoff_distance_brooklyn']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],brooklyn[1],brooklyn[0]),axis=1)
X_train['dropoff_distance_bronx']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],bronx[1],bronx[0]),axis=1)
X_train['dropoff_distance_statenisland']=X_train.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],staten_island[1],staten_island[0]),axis=1)




X_test['pickup_distance_manhattan']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],manhattan[1],manhattan[0]),axis=1)
X_test['pickup_distance_queens']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],queens[1],queens[0]),axis=1)
X_test['pickup_distance_brooklyn']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],brooklyn[1],brooklyn[0]),axis=1)
X_test['pickup_distance_bronx']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],bronx[1],bronx[0]),axis=1)
X_test['pickup_distance_statenisland']=X_test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],staten_island[1],staten_island[0]),axis=1)

X_test['dropoff_distance_manhattan']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],manhattan[1],manhattan[0]),axis=1)
X_test['dropoff_distance_queens']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],queens[1],queens[0]),axis=1)
X_test['dropoff_distance_brooklyn']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],brooklyn[1],brooklyn[0]),axis=1)
X_test['dropoff_distance_bronx']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],bronx[1],bronx[0]),axis=1)
X_test['dropoff_distance_statenisland']=X_test.apply(lambda row:distance(row['dropoff_latitude'],row['dropoff_longitude'],staten_island[1],staten_island[0]),axis=1)

In [42]:
X_train.to_csv("X_train_cleaned.csv",index=False)
X_test.to_csv("X_test_cleaned.csv",index=False)
test.to_csv("test_cleaned.csv",index=False)

In [43]:
train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':31, 'num_trees':5000, 'objective':'regression'}
param['metric'] = 'l2_root'

In [44]:
num_round=1000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=5,verbose_eval=20, early_stopping_rounds=20,stratified=False)

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:430: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	cv_agg's rmse: 4.97639 + 0.0433001
[40]	cv_agg's rmse: 4.40782 + 0.0470562
[60]	cv_agg's rmse: 4.19551 + 0.0508031
[80]	cv_agg's rmse: 4.08703 + 0.0437323
[100]	cv_agg's rmse: 4.02134 + 0.0462551
[120]	cv_agg's rmse: 3.97853 + 0.0481419
[140]	cv_agg's rmse: 3.94341 + 0.0470577
[160]	cv_agg's rmse: 3.91974 + 0.0477422
[180]	cv_agg's rmse: 3.9016 + 0.0487359
[200]	cv_agg's rmse: 3.88894 + 0.0484676
[220]	cv_agg's rmse: 3.8762 + 0.0489229
[240]	cv_agg's rmse: 3.86516 + 0.0487207
[260]	cv_agg's rmse: 3.85582 + 0.0485204
[280]	cv_agg's rmse: 3.84757 + 0.0480057
[300]	cv_agg's rmse: 3.83991 + 0.0491724
[320]	cv_agg's rmse: 3.83403 + 0.0487574
[340]	cv_agg's rmse: 3.82654 + 0.0480774
[360]	cv_agg's rmse: 3.82131 + 0.0492133
[380]	cv_agg's rmse: 3.81708 + 0.0495516
[400]	cv_agg's rmse: 3.81252 + 0.051389
[420]	cv_agg's rmse: 3.80834 + 0.0525033
[440]	cv_agg's rmse: 3.80426 + 0.0524189
[460]	cv_agg's rmse: 3.80059 + 0.0532246
[480]	cv_agg's rmse: 3.79669 + 0.053099
[500]	cv_agg's rmse: 3.7

In [45]:
print('Best num_boost_round:', len(cv_results['rmse-mean']))

Best num_boost_round: 1737


In [46]:
lgb_bst=lgb.train(param,train_data,len(cv_results['rmse-mean']))

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [47]:
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM with Feature Engineering is ",lgb_rmse)

RMSE for Light GBM with Feature Engineering is  3.6794217823934643


In [48]:
lgb_train_rmse=np.sqrt(mean_squared_error(lgb_bst.predict(X_train),y_train))
print("Train RMSE for Light GBM with Feature Engineering is", lgb_train_rmse)


Train RMSE for Light GBM with Feature Engineering is 2.8034321159702484


In [49]:
variance=lgb_train_rmse - lgb_rmse
print("Variance of Light GBM with Feature Engineering is ", variance)

Variance of Light GBM with Feature Engineering is  -0.8759896664232159
